In [1]:
import pylhe
import ROOT
import math
import os
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.20/04


In [2]:
##### setting file path #####

sig_rootpath = "/data/Chen-Wang/Nsubjettiness_test/Events/run_01/tag_1_delphes_events.root"
sig_file = uproot.open(sig_rootpath)

In [3]:
##### check final state electron(muon) in signal file #####

sig_electron = sig_file["Delphes;1"]["Electron"].array()
sig_muon = sig_file["Delphes;1"]["Muon.Particle"].array()
sig_photon = sig_file["Delphes;1"]["Photon"].array()
sig_photon_particles = sig_file["Delphes;1"]["Photon.Particles"].array()
sig_jet = sig_file["Delphes;1"]["Jet"].array()
sig_jet_particles = sig_file["Delphes;1"]["Jet.Particles"].array()
sig_jet_theta = sig_file["Delphes;1"]["Jet.EhadOverEem"].array()
sig_jet_mass = sig_file["Delphes;1"]["Jet.Mass"].array()
sig_num = sig_file["Delphes;1"]["Event.Number"].array()
sig_Nsubjettiness = sig_file["Delphes;1"]["Jet.Tau[5]"].array()

In [13]:
num = 4
print("In the", num, "th signal event:\n")
print("number of electron: ", sig_electron[num])
print("number of muon: ", sig_muon[num])
print("number of photon: ", sig_photon[num])
print("photon particles:", sig_photon_particles[num])
print("number of jets:", sig_jet[num])
print("jet particles:", sig_jet_particles[num])
print("jet theta:", sig_jet_theta[num])
print("jet mass:", sig_jet_mass[num])
print("event number:", sig_num[num])
print("event N-subjettiness:", sig_Nsubjettiness[num])

In the 4 th signal event:

number of electron:  2
number of muon:  []
number of photon:  0
photon particles: []
number of jets: 1
jet particles: [[267, 268]]
jet theta: [0.]
jet mass: [9.6934595]
event number: [4]
event N-subjettiness: [[0.06296376 0.         0.         0.         0.        ]]


In [14]:
##### set signal event content #####

events = [sig_file["Delphes;1"]["Particle.Status"].array(),
        sig_file["Delphes;1"]["Particle.M1"].array(),
        sig_file["Delphes;1"]["Particle.M2"].array(),
        sig_file["Delphes;1"]["Particle.D1"].array(),
        sig_file["Delphes;1"]["Particle.D2"].array(),
        sig_file["Delphes;1"]["Particle.PID"].array(),
        sig_file["Delphes;1"]["Particle.PT"].array(),
        sig_file["Delphes;1"]["Particle.Eta"].array(),
        sig_file["Delphes;1"]["Particle.Phi"].array(),
        sig_file["Delphes;1"]["Particle.Mass"].array()
        ]

##### reshape the signal data #####

events = np.expand_dims(events, axis=-1)
events = events.transpose((1,0,2))
events = np.squeeze(events,axis=(2,))

##### set features #####

_Status, _M1, _M2, _D1, _D2, _PID, _PT, _Eta, _Phi, _Mass = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 
Labels = ["Status", "M1", "M2", "D1", "D2", "PID", "PT", "Eta", "Phi", "Mass"]

##### Show truth record #####

print("Index", "\t","Status", "\t","M1"
    "\t","M2" ,"\t","D1", "\t","D2", "\t","PID", 
    "\t\t","PT" "\t","Eta", "\t\t","Phi", 
    "\t\t","Mass")

event_num = 4
for j in range(len(events[event_num][0])):
    print(j, "\t", events[event_num][_Status][j],"\t\t",
    events[event_num][_M1][j], "\t", events[event_num][_M2][j],
    "\t", events[event_num][_D1][j], "\t", events[event_num][_D2][j],
    "\t", str(events[event_num][_PID][j]).ljust(12, ' '), "\t", round(events[event_num][_PT][j],1),  "\t",
        str(round(events[event_num][_Eta][j],2)).ljust(12, ' ') , "\t",
        str(round(events[event_num][_Phi][j],3)).ljust(12, ' '), "\t",
        round(events[event_num][_Mass][j],3))

Index 	 Status 	 M1	 M2 	 D1 	 D2 	 PID 		 PT	 Eta 		 Phi 		 Mass
0 	 21 		 5 	 -1 	 3 	 4 	 -1           	 0.0 	 999.9        	 0.0          	 0.0
1 	 43 		 5 	 -1 	 14 	 14 	 21           	 15.6 	 2.29         	 2.621        	 0.0
2 	 21 		 6 	 -1 	 3 	 4 	 1            	 0.0 	 -999.9       	 0.0          	 0.0
3 	 22 		 0 	 2 	 8 	 8 	 23           	 159.9 	 0.86         	 2.705        	 91.031
4 	 22 		 0 	 2 	 9 	 9 	 9000005      	 159.9 	 -1.7         	 -0.436       	 10.0
5 	 41 		 10 	 -1 	 0 	 1 	 -1           	 0.0 	 999.9        	 2.356        	 0.0
6 	 42 		 11 	 -1 	 2 	 2 	 1            	 0.0 	 -999.9       	 2.554        	 0.0
7 	 43 		 11 	 -1 	 15 	 16 	 21           	 9.9 	 -2.36        	 1.574        	 0.0
8 	 44 		 3 	 -1 	 12 	 12 	 23           	 145.5 	 0.96         	 2.714        	 91.031
9 	 44 		 4 	 -1 	 13 	 13 	 9000005      	 161.0 	 -1.71        	 -0.437       	 10.0
10 	 42 		 98 	 -1 	 5 	 5 	 -1           	 0.0 	 999.9        	 1.574        	 0.0
11 	

In [6]:
##### show the branch in Delphes;1 #####

sig_file["Delphes;1"].show()

Event                      TStreamerInfo              asdtype('>i4')
Event.fUniqueID            TStreamerBasicType         asjagged(asdtype('>u4'))
Event.fBits                TStreamerBasicType         asjagged(asdtype('>u4'))
Event.Number               TStreamerBasicType         asjagged(asdtype('>i8'))
Event.ReadTime             TStreamerBasicType         asjagged(asdtype('>f4'))
Event.ProcTime             TStreamerBasicType         asjagged(asdtype('>f4'))
Event.ProcessID            TStreamerBasicType         asjagged(asdtype('>i4'))
Event.MPI                  TStreamerBasicType         asjagged(asdtype('>i4'))
Event.Weight               TStreamerBasicType         asjagged(asdtype('>f4'))
Event.CrossSection         TStreamerBasicType         asjagged(asdtype('>f4'))
Event.CrossSectionError    TStreamerBasicType         asjagged(asdtype('>f4'))
Event.Scale                TStreamerBasicType         asjagged(asdtype('>f4'))
Event.AlphaQED             TStreamerBasicType         asjagged

In [15]:
np.sqrt(0.15**2 + 0.082**2)

0.1709502851708648